In [98]:
#generate i2c bus traffic for developing i2c slave

import math

SDA_stream=""
SCL_stream=""

#timing in ns for 100khz i2c
TimeBit=250 # time for one bit in ns
TimeRise=1000
TimeFall=300
TimeHigh=4000
TimeLow=4700
TimeStart=4000 # start and stop bit timing

#DeviceAddress=int('0110000',2)
DeviceAddress=int('1010101',2)

# print bits scaled and stuffed for SDA
def scalebits(number,width,stuffing):
    ss=""
    s=format(number, '#010b')[2:]
    front=stuffing//2
    back=stuffing-front
    # front and back is where you want to insert noise to test quality reading with noisy line
    for x in s:       # print out all bits 
        ss+=x*front
        ss+=x*width  #scaling each bit to fit correctly
        ss+=x*back
    return ss
    

# write to one register.
def writeone(RegAdr,Data):
    string=""
    width=(TimeHigh//TimeBit)+(TimeLow//TimeBit)
    totalwidth=((TimeRise+TimeHigh)//TimeBit)+((TimeFall+TimeLow)//TimeBit)
    #print(width,totalwidth)
    offset=(TimeLow//TimeBit)//2
    # construct lead in 
    string+=str('1')*((TimeFall+TimeLow)//TimeBit-TimeHigh//TimeBit) # leadin time while SCL is inactive
    string+=str('0')*(TimeHigh//TimeBit) #This marks S
    string+=str('0')*offset
    # now construct device address data R/W = 0
    string+=scalebits(DeviceAddress<<1,width,totalwidth-width)
    #artificial ack
    string+=str('0')*totalwidth
    #write register address
    string+=scalebits(RegAdr,width,totalwidth-width)
    #artificial ack
    string+=str('0')*totalwidth
    #write data
    string+=scalebits(Data,width,totalwidth-width)
    #artificial ack
    string+=str('0')*totalwidth
    return string
    
#generate SCL by time settings
def genSCLtime(length):
    b=1
    string=""
    for i in range(0,length):
        string+=str('1')*((TimeRise+TimeHigh)//TimeBit)+str('0')*((TimeFall+TimeLow)//TimeBit)
    return string

# generate P at end of a session
def genP():
    sda=str('0')*(TimeHigh//TimeBit)
    sda+=str('1')*(TimeLow//TimeBit)
    scl=str('1')*((TimeHigh+TimeLow)//TimeBit)
    return sda,scl

#leadin for both SCL and SDA
#SCL_stream=str('1')*44
#SDA_stream=str('1')*44
# genereate test signals
SDA_stream+=writeone(0b10101010,0b10101010)

SDA_size=len(SDA_stream)
SCL_ticks=math.ceil(SDA_size/((TimeRise+TimeFall+TimeHigh+TimeLow)//TimeBit))
SCL_stream+=genSCLtime(SCL_ticks)

# end with P
SDA_t,SCL_t=genP()
SDA_stream+=SDA_t
SCL_stream+=SCL_t



#lead out for SCL and SDA
SCL_stream+=str('1')*44
SDA_stream+=str('1')*44

print("SDA size="+str(SDA_size)+" SCL ticks="+str(SCL_ticks))

print(SDA_stream)
print("SCL")

print(SCL_stream)

SDA size=1109 SCL ticks=28
1111000000000000000000000000011111111111111111111111111111111111111110000000000000000000000000000000000000000111111111111111111111111111111111111111100000000000000000000000000000000000000001111111111111111111111111111111111111111000000000000000000000000000000000000000011111111111111111111111111111111111111110000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111110000000000000000000000000000000000000000111111111111111111111111111111111111111100000000000000000000000000000000000000001111111111111111111111111111111111111111000000000000000000000000000000000000000011111111111111111111111111111111111111110000000000000000000000000000000000000000000000000000000000000000000000000000000011111111111111111111111111111111111111110000000000000000000000000000000000000000111111111111111111111111111111111111111100000000000000000000000000000000000000001111111111111111111111111111111111111111000000000000000000000000

In [90]:
SDA=""
SCL=""
def genP():
    sda=str('0')*(TimeHigh//TimeBit)
    sda+=str('1')*(TimeLow//TimeBit)
    scl=str('1')*((TimeHigh+TimeLow)//TimeBit)
    return sda,scl
print(genP())

('0000000000000000111111111111111111', '1111111111111111111111111111111111')
